In [2]:
import pandas as pd

data = pd.read_csv('data_kaggle.csv')

In [4]:
data['Date'] = pd.to_datetime(data['Date'], dayfirst=True)
data.set_index(['Date'], inplace=True)
data['Promo'] = data['Promo'].fillna(0)
data.reset_index().set_index(['SKU_id', 'Store_id', 'Date'], inplace=True)
data['Regular_Price'] = data['Regular_Price'].ffill().bfill()
data.reset_index().set_index(['Date'], inplace=True)

# add actual price (promo price when promo occurred or regular price otherwise)

data['Actual_Price'] = data['Promo_Price'].combine_first(data['Regular_Price'])
data['Promo_percent'] = (1 - (data['Actual_Price'] / data['Regular_Price']))
data = data.drop('Promo_Price', axis=1)

data.reset_index(inplace=True)
data["weekday"] = data['Date'].dt.weekday
data["monthday"] = data['Date'].dt.day
data['is_weekend'] = data['weekday'].isin([5, 6]) * 1
data['month_period'] = 0
data.loc[data['monthday'] >= 15, 'month_period'] = 1

data.head()

,Date,Store_id,SKU_id,Promo,Demand,Regular_Price,Actual_Price,Promo_percent,weekday,monthday,is_weekend,month_period
0,2015-01-01,1,1,0.0,22.0,163.78,163.78,0.0,3,1,0,0
1,2015-01-01,19,1,0.0,25.0,163.78,163.78,0.0,3,1,0,0
2,2015-01-01,39,2,0.0,3.0,135.78,135.78,0.0,3,1,0,0
3,2015-01-01,40,2,0.0,0.0,135.78,135.78,0.0,3,1,0,0
4,2015-01-01,18,1,0.0,13.0,163.78,163.78,0.0,3,1,0,0


In [5]:
data.to_csv('data_preprocessed.csv', index=False)